In [1]:
import pandas as pd


#only take 1367 rows randomly from the dataset
df = pd.read_csv(r'G:\Diksha\7 sem\finalyrpro\NLP-text-doc-classification\Dataset\combined_final_dataset.csv')
df = df.sample(n=1367, random_state=1)
df.to_csv('combined_final_dataset_train.csv', index=False)


#only take 100 rows randomly from the dataset
df = pd.read_csv(r'G:\Diksha\7 sem\finalyrpro\NLP-text-doc-classification\Dataset\combined_final_dataset.csv')
df = df.sample(n=100, random_state=1)
df.to_csv('combined_final_dataset_test.csv', index=False)


df_train = pd.read_csv('combined_final_dataset_train.csv')
df_test = pd.read_csv('combined_final_dataset_test.csv')


#drop columns Title. category and date
df_test = df_test.drop(['Title','category','date'],axis = 1)
df_train = df_train.drop(['Title','category','date'],axis = 1)

# Assuming df_train and df_test are your DataFrames
encoded_dict = {
    'News': 0,
    'Research Paper': 1,
    'Code': 2,
    'Medical': 3,
    'Legal': 4,
    'Financial documents': 5
}

df_train['main_category'] = df_train['main_category'].map(encoded_dict)
df_test['main_category'] = df_test['main_category'].map(encoded_dict)

from tensorflow.keras.utils import to_categorical
from keras.utils import to_categorical

y_train = to_categorical(df_train.main_category)
y_test = to_categorical(df_test.main_category)

import transformers
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')

#datatypes of description
df_train.description.dtype
#convert desription column to string
df_train['description'] = df_train['description'].astype(str)
df_test['description'] = df_test['description'].astype(str)

# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=df_train.description.tolist(),
    add_special_tokens=True,
    max_length=300,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=df_test.description.tolist(),
    add_special_tokens=True,
    max_length=300,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']


import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense

max_len = 300
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(6,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    clipnorm=1.0)
# Set loss and metrics
loss =CategoricalCrossentropy(from_logits = True)
metric = CategoricalAccuracy('balanced_accuracy'),
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = loss, 
    metrics = metric)

train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=1,
    batch_size=36
)

predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

import numpy as np
y_predicted = np.argmax(predicted_raw, axis = 1)
y_true = df_test.main_category

from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted))



Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w


4/4 [==============================] - 45s 9s/step
              precision    recall  f1-score   support

           0       0.95      0.95      0.95        19
           1       1.00      1.00      1.00         3
           2       1.00      1.00      1.00         3
           3       1.00      1.00      1.00        20
           4       1.00      1.00      1.00        28
           5       0.96      0.96      0.96        27

    accuracy                           0.98       100
   macro avg       0.99      0.99      0.99       100
weighted avg       0.98      0.98      0.98       100



In [2]:
import os
import shutil
import logging
from dir_ops import Move
import pandas as pd
from collections import defaultdict
from SearchAlgo import DocumentSearch
from transformers import AutoTokenizer,TFBertModel #importing the tokenizer and bert model
#import keras
import tensorflow as tf
from keras.models import load_model
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
bert = TFBertModel.from_pretrained('bert-base-cased')


#model = load_model("D:/Backup/Desktop/programs/NLP-text-doc-classification/customized_transformer_model.h5",custom_objects={"TFBertModel":TFBertModel})

'''#validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
for key , value in zip(encoded_dict.keys(),validation[0]):
    print(key,value)
    #ssvae the value ina  variable
    dict1 = {key:value}
'''
 


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

"#validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100\nfor key , value in zip(encoded_dict.keys(),validation[0]):\n    print(key,value)\n    #ssvae the value ina  variable\n    dict1 = {key:value}\n"

In [4]:
ex_file = r"G:\Diksha\7 sem\finalyrpro\NLP-text-doc-classification\demo\news doc.txt"

#extract text from file and store in a variable
with open(ex_file, 'r') as f:
    text = f.read()
print(text)
encoded_dict = {
    'News': 0,
    'Research Paper': 1,
    'Code': 2,
    'Medical': 3,
    'Legal': 4,
    'Financial documents': 5
}

x_val = tokenizer(
    text=text,
    add_special_tokens=True,
    max_length=300,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
#validation = model.predict({'input_ids':x_val['input_ids'],'attention_mask':x_val['attention_mask']})*100
for key , value in zip(encoded_dict.keys(),validation[0]):
   print(key,value)
#save the value ina  variable
   dict1 = {key:value}

On the solemn anniversary of 26/11, the world reflects on the tragic events that unfolded in Mumbai, India, in 2008. This day marks the 13th anniversary of the coordinated terrorist attacks that shook the nation, claiming the lives of 166 people and injuring hundreds more. The attacks targeted multiple locations, including the iconic Taj Mahal Palace Hotel, the Oberoi Trident Hotel, and the Chhatrapati Shivaji Terminus. Commemorative events, memorial services, and tributes are held to honor the victims and salute the resilience of the city. The legacy of 26/11 serves as a stark reminder of the persistent global challenges posed by terrorism and underscores the importance of international cooperation in combating extremism and ensuring the safety and security of communities worldwide
1/1 [==============================] - 1s 724ms/step
News 87.49363
Research Paper 31.51227
Code 32.221413
Medical 22.177235
Legal 44.891598
Financial documents 64.0305


In [7]:
#get maximum value in the dict
max_key = max(dict1, key=dict1.get)
print(max_key)


Financial documents


In [6]:
#select the key with the highest value
max_key = max(dict1, key=d).title()
print(max_key)
#max_key = "Research Paper"
mover = Move()
mover.classify_document(ex_file, max_key)

Financial Documents


In [ ]:
searcher = DocumentSearch()

user_keyword = input("comma separated keywords: ").split(",")
results = searcher.retrieve_documents(user_keyword)

In [ ]:
print("Retrieved Documents:")
for i, doc in enumerate(results, start=1,):
    print(f"{i}. Title: {doc['Title']}, Category: {doc['main_category']}")# Date: {doc['date']}, Description: x")

